#Продвинутый Python, ДЗ-2

Правила игры:

В домашке 8 задач, разбаловка указана в задании. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

В каждой задаче необходимо реализовать функцию, которая после будет проверяться через github classroom на тестах. Сами тесты лежит в гитхабе, можете локально проверить работу функций перед сдачей

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

В данной домашке нужно использовать pandas

In [1]:
# https://stackoverflow.com/a/60658965/7286121

from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

## Данные

В этом домашнем задании вам придется оказаться на месте аналитика в бразильском маркетплейсе [Olist](https://olist.com/pt-br/). Вам необходимо исследовать данные и на их основании сделать выводы, которые помогут бизнесу расцветать!

Данные находятся тут в файле archive.zip

In [2]:
#!pip install wget
#перед сдачей это закомментить
#Comment

In [3]:
# Код, который будет в каждом тесте, названия не менять
import pandas as pd
import numpy as np
import zipfile
import wget

url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
filename = wget.download(url)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('./')

customers = pd.read_csv('olist_customers_dataset.csv')
location = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
translation = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

Ван дано 9 датасетов, которые содержат в себе все данные по 100 000 заказам со всей Бразилии. Чтобы облегчить вам жизнь, вот связи по этим датасетам (файл product_category_name_translation является переводом названий категорий с португальского на английский)

![](https://i.imgur.com/HRhd2Y0.png)

Ну что же, начнем!

## Задание 1 (10 баллов)

Определите:

* Число товаров

* Среднюю стоимость товара (стоимость товара = среднее от цен в датасете items)

в разрезе категорий (все категории должны быть на английском языке)

Табличка, которая у вас должна получиться:

```
category | products | price

value    | value      | value
```

**Обратите внимание:**

Для категории portateis_cozinha_e_preparadores_de_alimentos перевод portable kitchen and food preparers

Для категории pc_gamer перевод PC Gamer

Для них нужно отдельно добавить перевод

In [4]:
%%write_and_run task_1.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_1(translation, items, products):
    res = pd.merge(products, items, how='left', on="product_id")
    res = pd.merge(res, translation, how='left', on="product_category_name")
    res["product_category_name_english"]=np.where(res["product_category_name"]=="portateis_cozinha_e_preparadores_de_alimentos", "portable kitchen and food preparers", res.product_category_name_english)
    res["product_category_name_english"]=np.where(res["product_category_name"]=="pc_gamer", "PC Gamer", res.product_category_name_english)

    res=res.groupby(["product_id", "product_category_name_english"]).agg({"price": "mean"}).reset_index()

    res=res.groupby(["product_category_name_english"]).agg({"product_id":"count", "price": "mean"}).reset_index()
    res=res.rename(columns={"product_category_name_english": "category", "product_id": "products"})
    
    return res

In [5]:
#Проверки

res = task_1(translation, items, products)

assert res[res.category == 'portable kitchen and food preparers'].price.values[0] == 186.996
assert len(res) == 73
assert len(res.drop_duplicates()) == 73
assert res[res.category == 'drinks'].products.values[0] == 81
assert res.products.sum() == 32341
assert res.price.sum() == 12459.751444351941
assert res[res.category == 'home_confort'].price.values[0] == 185.56926417326417

## Задание 2 (10 баллов)

Определите для каждого продавца основную категорию их продаж (категории должны быть на английском языке)

Табличка, которая у вас должна получиться:

```
seller_id | category

value    | value
```




In [6]:
%%write_and_run task_2.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_2(translation, products, items):
    res = pd.merge(items, products, how='left', on="product_id")
    res = pd.merge(res, translation, how='outer', on="product_category_name")

    res["product_category_name_english"]=np.where(res["product_category_name"]=="portateis_cozinha_e_preparadores_de_alimentos", "portable kitchen and food preparers", res.product_category_name_english)
    res["product_category_name_english"]=np.where(res["product_category_name"]=="pc_gamer", "PC Gamer", res.product_category_name_english)

    res=res.groupby(["seller_id", "product_category_name_english"]).agg({"order_id":"count"}).reset_index().sort_values(by=["order_id"], ascending=False)
    res=res.drop_duplicates(subset=["seller_id"], keep="first").reset_index()

    res=res[["seller_id", "product_category_name_english"]]
    res=res.rename(columns={"product_category_name_english": "category"})
    return res

In [7]:
# Проверки

res = task_2(translation, products, items)
assert res[res.seller_id == 'e3e15e2c0b9700561efac21c6be48066'].category.values[0] == 'housewares'
assert res[res.seller_id == '2f73e04d12cdf0c945ded66bb3fcf6c7'].category.values[0] == 'garden_tools'
assert len(res) == len(res.drop_duplicates())
assert len(res) == 3035
assert len(res[res.category == 'telephony']) == 66
assert list(np.sort(res.groupby("category").agg({"seller_id": "nunique"}).seller_id.values)) == [  1,   1,   1,   1,   1,   2,   2,   2,   2,   2,   3,   3,   4,
         4,   4,   4,   5,   5,   5,   5,   5,   5,   6,   6,   6,   7,
         8,  10,  12,  13,  13,  13,  14,  14,  14,  15,  16,  17,  17,
        17,  19,  20,  20,  20,  21,  22,  26,  37,  37,  43,  46,  51,
        54,  59,  66,  66,  78,  87,  87,  99, 101, 116, 125, 156, 216,
       224, 256, 288, 310]

## Задание 3 (10 баллов)

Выведите долю потраченных денег по каждому штату (потраченные деньги - сумма денег по доставленным заказам, сумма по price и freight_value)

*Примечание:* разбивка по штатам - по штату покупателя, процент - число от 0 до 1


Табличка, которая у вас должна получиться:

```
state | perc

value | value
```

In [8]:
%%write_and_run task_3.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_3(orders, customers, items):
    deliv_orders=orders[orders["order_status"]=="delivered"]
    res = pd.merge(deliv_orders, items, how='inner', on="order_id")
    res = pd.merge(res, customers, how='inner', on="customer_id")

    res['total_sum'] = res[["price", "freight_value"]].sum(axis=1)

    res=res[["customer_id", "customer_state", "total_sum"]]
    res=res.groupby(["customer_state"]).agg({"total_sum":"sum"}).reset_index()
    overall_sum=res.total_sum.sum()
    res["total_sum"]=res["total_sum"].div(overall_sum)
    res=res.rename(columns={"customer_state":"state", "total_sum":"perc"})

    return res

In [9]:
# Проверки

res = task_3(orders, customers, items)
assert res.perc.sum() == 1
assert res[res.state == "RS"].perc.values[0] == 0.055868056429816286
assert res.sort_values("perc", ascending=True).iloc[0, 1] == 0.0005862290943146945
assert res.sort_values("perc", ascending=False).iloc[0, 1] == 0.3741756035817322
assert len(res) == 27

## Задание 4 (10 баллов)

Определите средний чек покупки (добавьте разбивку на стоимость самого заказ и стоимость доставки) и среднее число товаров в заказе

А также определите среднее число покупок на пользователя (обратите внимание на идентификаторы)

In [10]:
%%write_and_run task_4.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_4(items, orders, customers):
    mean_price=items.groupby(["order_id"]).agg({"price":"sum"}).reset_index()["price"].mean()
    mean_freight=items.groupby(["order_id"]).agg({"freight_value":"sum"}).reset_index()["freight_value"].mean()

    new_items=items.copy(deep=True)[["order_id", "order_item_id"]]
    new_items["cnt"]=1
    mean_quant=new_items.groupby(["order_id"]).agg({'cnt':"count"})['cnt'].mean()

    new_orders=pd.merge(orders, customers, how="inner", on="customer_id")
    mean_orders=new_orders.groupby(["customer_unique_id"]).agg({"order_id":"count"}).reset_index()["order_id"].mean()

    return (mean_price, mean_freight, mean_quant, mean_orders)

In [11]:
# Проверка

res = task_4(items, orders, customers)
assert res == (137.7540763788945, 22.823561713254815, 1.1417306873695092, 1.0348089410589412)

## Задание 5 (10 баллов)

Посчитайте CSAT (customer satisfaction - средняя оценка ревью) и отобразите средний CSAT по дням в период с апреля 2017 по апрель 2018 года

Все манипуляции с датой необходимо сделать с помощью datetime и dateutil


Табличка, которая у вас должна получиться:

```
date | csat

"YYYY-MM-DD" | value
```

In [12]:
%%write_and_run task_5.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd
from dateutil.parser import parse
import datetime

def task_5(reviews):
    res=reviews.copy(deep=True)[["review_score", "review_creation_date"]]
    res["review_creation_date"]=res["review_creation_date"].apply(lambda x: parse(x))
    res["review_creation_date"]=res["review_creation_date"].apply(lambda x: datetime.date(x.year, x.month, x.day))
    beg=parse("2017-04-01")
    end=parse("2018-04-30")
    res=res[(datetime.date(beg.year, beg.month, beg.day)<=res["review_creation_date"])&(res["review_creation_date"]<=datetime.date(end.year, end.month, end.day))].reset_index()

    res=res.groupby(["review_creation_date"]).agg({'review_score':"mean"}).reset_index()
    res["review_creation_date"]=res["review_creation_date"].apply(lambda x: x.strftime("%Y-%m-%d"))

    res=res.rename(columns={"review_creation_date":"date", "review_score":"csat"})

    return res

In [13]:
# Проверки
res = task_5(reviews)
assert res.date.min() == '2017-04-01'
assert res.date.max() == '2018-04-30'
assert res.csat.sum() == 1551.8881071384853
assert res[res.date == '2017-07-11'].csat.values[0] == 4.291390728476821
assert res[res.date == '2018-02-09'].csat.values[0] == 3.992156862745098
assert res[res.csat == 3.6814814814814816].date.values[0] == '2018-02-25'

## Задание 6 (10 баллов)

Посмотрите, как быстро отвечают пользователи (сделайте аггреграцию по числу дней ответа) и какая средняя оценка

Все манипуляции со временем нужно делать через datetime и dateutil

Табличка, которая у вас должна получиться:

```
days | csat | orders

value | value | value
```

Результаты должны быть отсориртированы по дня по возрастанию

In [14]:
%%write_and_run task_6.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd
from dateutil.parser import parse
import datetime

def task_6(reviews):
    res=reviews.copy(deep=True)[["review_score", "order_id", "review_creation_date", "review_answer_timestamp"]]
    res['review_answer_timestamp']=res["review_answer_timestamp"].apply(lambda x: parse(x))
    res['review_creation_date']=res["review_creation_date"].apply(lambda x: parse(x))
    res["days"]=res["review_answer_timestamp"]-res["review_creation_date"]
    res['days']=res["days"].apply(lambda x: x.days)
    res=res.groupby(["days"]).agg({'review_score':"mean", "order_id":"count"}).reset_index()
    res=res.rename(columns={"review_score":"csat", "order_id":"orders"})

    return res

In [15]:
# Проверки
res = task_6(reviews)
assert res.orders.sum() == 99224
assert np.all(res.days.values == np.sort(res.days.values))
assert len(res) == 214
assert res.days.min() == 0
assert res.days.max() == 518
assert res[res.days == 233].csat.values[0] == 3.0
assert res[res.days == 87].orders.values[0] == 4

## Задание 7 (10 баллов)

Выделите все заказы, где не проставлено поле order_delivered_customer_date. Замените его на дату '2999-12-31'

In [16]:
%%write_and_run task_7.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_7(orders):
    res=orders.copy(deep=True)
    res["order_delivered_customer_date"]=res["order_delivered_customer_date"].fillna('2999-12-31')
    return res

In [17]:
#Проверки

res = task_7(orders)
assert len(res[res.order_delivered_customer_date.isna()]) == 0
assert len(res) == 99441
assert len(res[res.order_delivered_customer_date == '2999-12-31']) == 2965

## Задание 8 (30 баллов)

Определите топ-10 продавцов, у которых больше 100 заказов, отсылающие чаще всего свою посылку в другие регионы (считаются только доставленные заказы)

Чаще всего отсылают = самый большой процент отправленных заказов в другой штат

Табличка, которая у вас должна получиться:

```
seller_id | share

value | value
```

In [18]:
%%write_and_run task_8.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_8(orders, items, sellers, customers):
    new_orders=orders[orders["order_status"]=="delivered"]
    res = pd.merge(new_orders, items, how='left', on="order_id")
    res = pd.merge(res, customers, how='left', on="customer_id")
    res = pd.merge(res, sellers, how='left', on="seller_id")
    res=res[["order_id", "order_item_id", "seller_id", "product_id", "seller_state", "customer_state"]]
    res=res.drop_duplicates(subset=["order_id", "seller_id"])

    new=res.groupby(["order_id", "product_id"]).agg({"seller_state":"count"}).reset_index()
    res["all_orders"]=1
    res["foreign_orders"]=(res["seller_state"]!=res["customer_state"])
    res["foreign_orders"]=res["foreign_orders"].apply(lambda x: int(x))

    res=res.groupby(["seller_id"]).agg({"all_orders":"sum", "foreign_orders":"sum"}).reset_index()
    res=res[res["all_orders"]>100]
    res["share"]=res["foreign_orders"]/res["all_orders"]
    res=res.sort_values(["share"], ascending=False)
    res=res.iloc[:10].reset_index()
    res=res[["seller_id", "share"]]

    return res

In [19]:
# Проверки

res = task_8(orders, items, sellers, customers)
assert np.all(res.share.values == np.sort(res.share.values)[::-1])
assert res.share.values[0] == 0.9743589743589743
assert res.share.values[-1] == 0.9356435643564357
assert res.seller_id.values[5] == '1b4c3a6f53068f0b6944d2d005c9fc89'
assert res.seller_id.values[2] == '06a2c3af7b3aee5d69171b0e14f0ee87'
assert res.share.sum() == 9.519118744616716